In [2]:
import os

In [3]:
%pwd

'c:\\Users\\User\\Desktop\\PROJECTS\\stock\\Power-Time-Series-App\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\User\\Desktop\\PROJECTS\\stock\\Power-Time-Series-App'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from powerTimeSeries.constants import *
from powerTimeSeries.utils.common import read_yaml, create_directories

ModuleNotFoundError: No module named 'powerTimeSeries'

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [12]:
import os
import urllib.request as request
import zipfile
from studentApp import logger
from studentApp.utils.common import get_size

In [13]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-07-17 18:52:48,711: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-17 18:52:48,725: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-17 18:52:48,732: INFO: common: created directory at: artifacts]
[2023-07-17 18:52:48,734: INFO: common: created directory at: artifacts/data_ingestion]
[2023-07-17 18:52:50,729: INFO: 1170291011: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7819
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "7f6ff80444316700b62799c4e98bfbef97c0c5ef82c560a393c976f0758b92ec"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 9E68:3253:A453F:F8FA0:64B56000
Accept-Ranges: bytes
Date: Mon, 17 Jul 2023 15:52:50 GMT
Via: 1.1 varnish
X-Served-By: cache-jnb7025-JNB
X-Cache: HIT
X-Cac